In [3]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [6]:
# Let's first get some files that we want to use
doc_dir = "data/"

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\1-aisys.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\10_logiq.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\11_csdiagnostic.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\12_mac2000.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\2_9100cnxt.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\3_mac5500.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\4_seer100.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\5_senopristina.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\6_senobright.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting data\7_discovery656.txt
11/27/2020 19:26:34 - INFO - haystack.preprocessor.utils -   Converting 

In [ ]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

11/27/2020 19:26:51 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/27/2020 19:26:56 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/27/2020 19:26:57 - INFO - elasticsearch -   POST http://localhost:9200/rethinkl_test1/_search?scroll=5m&size=1000 [status:200 request:0.018s]
11/27/2020 19:26:57 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.035s]
11/27/2020 19:26:57 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.000s]
11/27/2020 19:26:57 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for 12 docs ...
Inferenc

In [5]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="rethinkl_test1")

11/27/2020 19:26:27 - INFO - elasticsearch -   PUT http://localhost:9200/rethinkl_test1 [status:200 request:0.250s]
11/27/2020 19:26:27 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.016s]


In [3]:
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

NameError: name 'doc_dir' is not defined

In [6]:
document_store.write_documents(dicts)

11/23/2020 08:22:04 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.350s]
11/23/2020 08:22:05 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.114s]
11/23/2020 08:22:06 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.092s]
11/23/2020 08:22:07 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.059s]
11/23/2020 08:22:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.061s]


In [4]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [30]:
document_store.query(query="Who is Arya Stark?", filters=None, top_k=5)

11/23/2020 11:36:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.030s]


In [6]:
context_list = retriever.retrieve(query="Who is Arya Stark?", top_k=5)

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.15 Batches/s]
11/27/2020 05:58:38 - INFO - elasticsearch -   POST http://localhost:9200/new_docs/_search [status:200 request:0.124s]


In [7]:
this = listed[1].to_dict()

NameError: name 'listed' is not defined

In [8]:
context = ''
for i in range(len(context_list)):
    context_list[i] = context_list[i].to_dict()['text'].replace("\n", " ")
    context += context_list[i] + ' '

In [9]:
context

' === Catelyn Stark === Catelyn Stark is the Lady of Winterfell, wife of Lord Eddard Stark, and mother to his children Robb, Sansa, Arya, Bran, and Rickon. She is the daughter of Lord Hoster Tully of Riverrun; niece to Ser Brynden Tully (also known as the legendary "Blackfish") and sister to Lysa Arryn of the Vale and to Edmure Tully. In the HBO television adaptation, she is portrayed by Michelle Fairley.  === Background === Arya is the third child and younger daughter of Eddard and Catelyn Stark and is nine years old at the beginning of the book series.  She has five siblings: an older brother Robb, an older sister Sansa, two younger brothers Bran and Rickon, and an older illegitimate half-brother, Jon Snow. \'\'\'Arya Stark\'\'\' is a fictional character in American author George R. R. Martin\'s \'\'A Song of Ice and Fire\'\' epic fantasy novel series.  She is a prominent point of view character in the novels with the third most viewpoint chapters, and is the only viewpoint character

In [1]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.dense import DensePassageRetriever

In [2]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="new_docs")
retriever = DensePassageRetriever(document_store=document_store)

11/27/2020 14:57:55 - INFO - elasticsearch -   HEAD http://localhost:9200/new_docs [status:200 request:0.017s]
11/27/2020 14:57:55 - INFO - elasticsearch -   GET http://localhost:9200/new_docs [status:200 request:0.000s]
11/27/2020 14:57:55 - INFO - elasticsearch -   PUT http://localhost:9200/new_docs/_mapping [status:200 request:0.016s]
11/27/2020 14:57:55 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.000s]
11/27/2020 14:58:02 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
11/27/2020 14:58:09 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [3]:
context_list = retriever.retrieve(query="Who is Arya Stark?", top_k=5)

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.87 Batches/s]
11/27/2020 14:58:45 - INFO - elasticsearch -   POST http://localhost:9200/new_docs/_search [status:200 request:0.265s]


In [4]:
from haystack.generator.transformers import RAGenerator

In [5]:
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=False,
    top_k_answers=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

11/27/2020 15:00:21 - INFO - filelock -   Lock 2172311605192 acquired on C:\Users\gemma/.cache\torch\transformers\6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1.lock


11/27/2020 15:00:21 - INFO - filelock -   Lock 2172311605192 released on C:\Users\gemma/.cache\torch\transformers\6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1.lock


OSError: Can't load tokenizer for 'facebook/rag-token-nq\question_encoder_tokenizer'. Make sure that:

- 'facebook/rag-token-nq\question_encoder_tokenizer' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'facebook/rag-token-nq\question_encoder_tokenizer' is the correct path to a directory containing relevant tokenizer files

